In [1]:
import json

train_file = '/home/vietchu/paraphrase-t5/data/paws_qqp_train_full.json'
test_file = '/home/vietchu/paraphrase-t5/data/paws/test_full.json'

with open(train_file, 'r') as f:
    train_data = json.load(f)
with open(test_file, 'r') as f:
    test_data = json.load(f)

In [18]:
from vncorenlp import VnCoreNLP
rdrsegmenter = VnCoreNLP("/home/vietchu/VnCoreNLP/VnCoreNLP-1.1.1.jar", annotators="wseg", max_heap_size='-Xmx500m')

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [74]:
for i, data in enumerate(train_data):
    train_data[i]['Source'] = ' '.join(rdrsegmenter.tokenize(data['Source'])[0])
    train_data[i]['Target'] = ' '.join(rdrsegmenter.tokenize(data['Target'])[0])

In [77]:
for i, data in enumerate(test_data):
    test_data[i]['Source'] = ' '.join(rdrsegmenter.tokenize(data['Source'])[0])
    test_data[i]['Target'] = ' '.join(rdrsegmenter.tokenize(data['Target'])[0])

In [83]:
with open(train_file, 'w') as f:
    json.dump(train_data, f)
with open(test_file, 'w') as f:
    json.dump(test_data, f)

## START HERE

In [240]:
from vncorenlp import VnCoreNLP
rdrsegmenter = VnCoreNLP("/home/vietchu/VnCoreNLP/VnCoreNLP-1.1.1.jar", annotators="wseg", max_heap_size='-Xmx500m') 

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [241]:
import os
from transformers import MBartTokenizerFast, MBartForConditionalGeneration, AutoTokenizer
from transformers import MT5ForConditionalGeneration, MT5Tokenizer
import pytorch_lightning as pl
import torch
from torch.utils.data import TensorDataset, random_split
from transformers.optimization import AdamW
from pytorch_lightning.callbacks import Callback
from tqdm import tqdm

MAX_SOURCE_LENGTH, MAX_TARGET_LENGTH = 200, 200

In [242]:
epoch_num = 9
model = MBartForConditionalGeneration.from_pretrained(f"../models/bartpho_paws_qqp_segmented/epoch_{epoch_num}")
# 5 is best from 1 to 6
tokenizer = AutoTokenizer.from_pretrained("vinai/bartpho-word")

In [243]:
device = torch.device('cpu')

def main(sent, num_outputs: int = 8, max_length: int = 200):
    generated = model.generate(
        tokenizer.encode(sent, max_length=max_length, truncation=True, padding='max_length', return_tensors='pt').to(device),
        num_beams=20, num_return_sequences=num_outputs, max_length=max_length
    )

    result = []
    for generated_sentence in generated:
        out = tokenizer.decode(
                generated_sentence,
                skip_special_tokens=True
            )
        result.append(out)
    return result

In [244]:
def levenshteinDistance(s1, s2):
    s1 = s1.split()
    s2 = s2.split()
    if len(s1) > len(s2):
        s1, s2 = s2, s1

    distances = range(len(s1) + 1)
    for i2, c2 in enumerate(s2):
        distances_ = [i2+1]
        for i1, c1 in enumerate(s1):
            if c1 == c2:
                distances_.append(distances[i1])
            else:
                distances_.append(1 + min((distances[i1], distances[i1 + 1], distances_[-1])))
        distances = distances_
    final_distance = distances[-1]
    score = final_distance / max(len(s1), len(s2))
    return {
        "raw_distance": final_distance,
        "score": score
    }

def filter_answer(raw_sentence, paraphrase_sentences):
    output = []
    for sen in paraphrase_sentences:
        distance_score = levenshteinDistance(raw_sentence, sen)
        raw_distance = distance_score["raw_distance"]
        score = distance_score["score"]
        if raw_distance >= 0.02*len(raw_sentence):
            output.append((raw_distance, sen))
    output.sort(reverse=True)
    return output

In [257]:
import pickle
with open('./data', 'rb') as f:
    manual_data = pickle.load(f) 
manual_data = manual_data[1000:1998]

for i, dat in enumerate(manual_data):
    manual_data[i] = dat['message']

In [258]:
manual_data[0]

' Phớt_lờ các quy_định của pháp_luật về hành_vi đổi tiền_lẻ , thị_trường chợ_đen online đang chạy_đua , nhiễu_loạn đủ mức phí đổi tiền . Không khó để tìm được một địa_chỉ nhận đổi tiền_lẻ , tiền mới , nguyên seri với những lời mời_chào " đổi bao_nhiêu và mệnh_giá nào cũng có " trên các trang Facebook , Fanpage hoặc website đổi tiền Tết 2020 . Điểm chung của những trang đổi tiền_lẻ này là đều đưa ra mức giá chênh_lệch đổi tiền rất cao , có chiết_khấu cho khách mua_sỉ hoặc công_ty cần mua số_lượng lớn , vận_chuyển tận nhà mới cần thanh_toán phí đổi . Cụ_thể , khách_hàng đổi mệnh_giá 5.000 – 10.000 đồng với số_lượng từ 100 – 1.000 tờ sẽ chịu mức phí 10-14% . Nếu khách đổi lẻ hoặc ít hơn thì phải chịu phí gấp đôi , tới 20% . Viện lý_do tiền_lẻ mệnh_giá dưới 1.000 đồng khan_hiếm hơn , nhiều nơi " hét " phí đổi tiền lên 270-300% . Tương_ứng với việc nếu đổi 1 triệu đồng tiền 500 đồng , khách_hàng phải chịu mức phí lên tới 3 triệu đồng . Ngoài việc cung_cấp các mệnh_giá thấp để đổi cho khách 

In [246]:
text0 = "Người ra đi đầu không ngoảnh lại, sau lưng thềm nắng lá rơi đầy." # this has to be 1 sentence only
text = rdrsegmenter.tokenize(text0)
for i, tex in enumerate(text): 
    text[i] = ' '.join(tex)

text = ' '.join(text)
paraphrases = main(text)

for i, _ in enumerate(paraphrases):
    paraphrases[i] = paraphrases[i].replace('_', ' ')

output = filter_answer(text0, paraphrases)

In [247]:
from sentence_transformers import SentenceTransformer, util
sentences = [text0] + output
for i, paraphrased_sentences in enumerate(sentences):
    print(sentences[i] if type(sentences[i]) == str else sentences[i][1])

Người ra đi đầu không ngoảnh lại, sau lưng thềm nắng lá rơi đầy.
Người ra đi đầu không ngoảnh lại, những người ở sau lưng thềm nắng rơi đầy.
Người ra đi đầu không ngoảnh lại, người ở sau lưng thềm nắng rơi đầy.
Người ra đi đầu không ngoảnh lại, sau lưng thềm nắng rụng đầy.
Người ra đi đầu không ngoảnh lại, phía sau lưng thềm nắng rơi đầy.
Người ra đi đầu không ngoảnh lại, nơi lưng thềm nắng rơi đầy.
Người ra đi đầu không ngoảnh lại, người ở sau lưng thềm nắng lá rơi đầy.


In [248]:
from xlwt import Workbook
wb = Workbook()
sheet1 = wb.add_sheet('Sheet 1')
l = 0
for index in range(0, len(manual_data), 20):
    text0 = manual_data[index]

    text = rdrsegmenter.tokenize(text0)
    for i, tex in enumerate(text): 
        text[i] = ' '.join(tex)
    text = ' '.join(text)

    paraphrases = main(text)
    for i, sentence in enumerate(paraphrases):
        paraphrases[i] = paraphrases[i].replace('_', ' ')

    output = filter_answer(text0, paraphrases)
    output = [text0] + output
    cell = ''
    for o in output:
        cell += (o[1] + '\n') if type(o) == tuple else (o + '\n')
    
    sheet1.write(l+1, 0, cell)
    l += 1
    print(cell)

wb.save(f'Paraphrase_test_{epoch_num}.xls')

Tụi tôi còn được học đan nữa, vui lắm, cỡ này không còn phải sang Campuchia bán bàng nữa ".
Tụi tôi còn được đan nữa, vui lắm.
Tụi tôi còn được học đan nữa, vui lắm.
Tụi tôi còn được học đan nữa, chắc chắn là vui lắm, cỡ này không còn phải sang Campuchia bán bàng nữa.
Tụi tôi còn được học đan nữa, rất vui, cỡ này không còn phải sang Campuchia bán bàng nữa.
Tụi tôi còn được học đan nữa, cảm thấy vui lắm, cỡ này không còn phải sang Campuchia bán bàng nữa.
Tụi tôi còn được đan nữa, vui lắm, cỡ này không còn phải sang Campuchia bán bàng nữa.
Tụi tôi còn được học đan nữa, chắc lắm, cỡ này không còn phải sang Campuchia bán bàng nữa.
Tụi tôi còn được học đan nữa, vui lắm, cỡ này không còn phải sang Campuchia bán bàng nữa.

Sau chuyến đi khảo sát ấy, các nhà khoa học lập tức kiến nghị Ubnd tỉnh Kiên Giang cho thực hiện dự án " Bảo tồn, sử dụng bền vững đồng cỏ bàng, kết hợp bảo tồn nghề thủ công truyền thống ".
Sau chuyến đi khảo sát ấy, các nhà khoa học đã yêu cầu Ubnd tỉnh Kiên Giang cho dự 